In [23]:
from main import Simulation

In [24]:
sim = Simulation()

/home/zsvizi/workspace/social-mixing-covid19-hungary/dataloader.py:72: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  data = pd.read_csv(self._contact_data_file, delimiter=',|:',
/home/zsvizi/workspace/social-mixing-covid19-hungary/dataloader.py:85: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  data = pd.read_csv(self._reference_contact_file, delimiter=',|:',
/home/zsvizi/workspace/social-mixing-covid19-hungary/dataloader.py:97: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex

In [25]:
# is_r_eff_calc:
# if True, then effective R is calculated, else R_0 is calculated
sim.is_r_eff_calc

False

In [26]:
# Call simulate() method to get r_eff values for the contact matrices
sim.simulate()

In [27]:
# r_eff_plot contains 1 + number_of_days * bin_size elements
# r_eff_plot[0] is calculated from contact_matrix/results/online_reference.csv
sim.r_eff_plot.shape

(2181,)

In [28]:
# If r_eff_calc is False, then for the kth day (k!=0), there are `bin_size` EQUAL numbers
# r_eff_plot[0] appears once
# E.g. for k=10
k = 10
sim.r_eff_plot[1 + k * sim.bin_size:(k+1) * sim.bin_size]

array([1.08602453, 1.08602453, 1.08602453, 1.08602453, 1.08602453,
       1.08602453, 1.08602453, 1.08602453, 1.08602453])

In [29]:
# For r_eff_calc = False, you can filter for unique values
daily_r_eff = sim.r_eff_plot[::sim.bin_size]
daily_r_eff.shape

(219,)

In [32]:
# Call get_repi_r0_list() to get r_eff values (here is_r_eff_calc is set to False)
# This method fills the member variable repi_r0_list
sim.get_repi_r0_list()
sim.repi_r0_list

-------- Representative matrices --------
Baseline beta: 0.12736156303151233
For matrix BASELINE eig. val = 17.2736573549719 -> baseline r0 = 2.2
-----------------------------------------
For matrix reference eig. val = 57.63074862280115 -> r0 = 7.3399422232761315
For matrix 04_2020 eig. val = 10.283749564221493 -> r0 = 1.3097544183238832
For matrix 05_2020 eig. val = 10.207891616336124 -> r0 = 1.3000930315128394
For matrix 06_2020 eig. val = 11.568319131809496 -> r0 = 1.473359206274605
For matrix 08_2020 eig. val = 12.475132761834743 -> r0 = 1.5888524075729002
For matrix 09_2020 eig. val = 17.012010113814355 -> r0 = 2.1666761984032923
For matrix 10_2020 eig. val = 20.379641518905828 -> r0 = 2.59558299786975
For matrix 11_2020 eig. val = 10.304569611004855 -> r0 = 1.3124060920246012
For matrix 12_2020 eig. val = 13.371338953128284 -> r0 = 1.702994628894564


[7.3399422232761315,
 1.3097544183238832,
 1.3000930315128394,
 1.473359206274605,
 1.473359206274605,
 1.5888524075729002,
 2.1666761984032923,
 2.59558299786975,
 1.3124060920246012,
 1.702994628894564]